In [63]:
import utils.fflow as flw
import numpy as np
import torch
import os
import multiprocessing
from bitsets import bitset
import random
from copy import deepcopy
import networkx as nx
import metis

In [2]:
multiprocessing.set_start_method('spawn')
option = {
    'task': 'synthetic_cnum16_non_iid_00_00',
    'algorithm': 'mp_fedavg',
    'model': 'lr',
    'sample': 'uniform',
    'aggregate': 'none',
    'learning_rate_decay': 0.998,
    'weight_decay': 0,
    'lr_scheduler': -1,
    'num_rounds': 0,
    'proportion': 1.0,
    'num_epochs': 0,
    'learning_rate': 0.1,
    'batch_size': 10,
    'optimizer': 'SGD',
    'momentum': 0,
    'seed': 0,
    'eval_interval': 1,
    'num_threads': 1,
    'num_threads_per_gpu': 8,
    'gpu': [0],
    'net_drop': 0,
    'net_active': 99999,
    'capability': 0,
    'learning_rate_lambda': 0,
    'q': 0.0,
    'epsilon': 0.0,
    'eta': 1.0,
    'tau': 0,
    'alpha': 0.0,
    'beta': 1.0,
    'gamma': 0.0,
    'mu': 0.1,
    'server_gpu_id': 0,
    'num_gpus': 1
}
option['num_gpus'] = len(option['gpu'])
os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(gpu_id) for gpu_id in option['gpu']])
os.environ['MASTER_ADDR'] = "localhost"
os.environ['MASTER_PORT'] = '8888'
os.environ['WORLD_SIZE'] = str(3)
# set random seed
flw.setup_seed(option['seed'])
# initialize server
server = flw.initialize(option)

init fedtask...done
init clients...done
init server...done


In [3]:
server.model.layer.weight.device

device(type='cuda', index=0)

In [4]:
ROUND = 1

In [5]:
server.global_save_dir, server.local_save_dir

('./chkpts/synthetic_cnum16_non_iid_00_00/global',
 './chkpts/synthetic_cnum16_non_iid_00_00/local')

In [6]:
ROUND_PARTICIPANTS_INDEXES = [
    int(file.replace('Client', '').replace('.pt', ''))
    for file in os.listdir(os.path.join(
        server.local_save_dir, 'Round{}'.format(ROUND)
    ))
]
ROUND_PARTICIPANTS_INDEXES.sort()
ROUND_PARTICIPANTS_INDEXES

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [7]:
ROUND_CLIENTS_BITSET = bitset('ROUND_CLIENTS_BITSET', tuple(ROUND_PARTICIPANTS_INDEXES))

In [8]:
DICT = dict()
ROUND_ALL_PARTICIPANTS_BITSET_KEY = ROUND_CLIENTS_BITSET(ROUND_PARTICIPANTS_INDEXES).bits()
print(ROUND_ALL_PARTICIPANTS_BITSET_KEY)
DICT[ROUND_ALL_PARTICIPANTS_BITSET_KEY] = 1
DICT

1111111111111111


{'1111111111111111': 1}

In [25]:
for _ in range(10):
    selected_clients_indexes = random.sample(ROUND_PARTICIPANTS_INDEXES, 3)
    bitset_key = ROUND_CLIENTS_BITSET(selected_clients_indexes).bits()
    DICT[bitset_key] = 1

In [26]:
[server.clients[i].name for i in selected_clients_indexes]

['Client13', 'Client15', 'Client8']

In [13]:
CPU = torch.device('cpu')
server.model.to(CPU)

Model(
  (layer): Linear(in_features=30, out_features=10, bias=True)
)

In [18]:
for client in server.clients:
    client.model = deepcopy(server.model)
    client.model.load_state_dict(
        torch.load(os.path.join(
            server.local_save_dir, 'Round{}/{}.pt'.format(ROUND, client.name)
        ))
    )

In [21]:
server.model.layer.bias, server.clients[0].model.layer.bias

(Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True),
 Parameter containing:
 tensor([-0.0645, -0.0979, -0.0010, -0.0659, -0.0641,  0.2343, -0.0803, -0.0970,
          0.1359,  0.1005], requires_grad=True))

In [32]:
selected_clients = [server.clients[index] for index in selected_clients_indexes]
models = [client.model for client in selected_clients]
p = np.array([client.datavol for client in selected_clients])
p = p / p.sum()
p

array([0.32454361, 0.26369168, 0.41176471])

In [37]:
server.model = server.aggregate(models=models, p=p)

In [39]:
server.model.layer.bias

Parameter containing:
tensor([-0.0054,  0.1488, -0.0369, -0.0396,  0.0256, -0.0267,  0.0272, -0.0419,
        -0.0348, -0.0163], requires_grad=True)

In [43]:
DICT = dict()

In [47]:
CUDA = torch.device('cuda')

In [48]:
server.model.to(CUDA)
server.test()

(0.2552083333333333, 2.1994385719299316)

In [50]:
server.model.load_state_dict(
    torch.load(
        os.path.join(server.global_save_dir, 'Round{}.pt'.format(ROUND)),
        map_location=CUDA
    )
)

<All keys matched successfully>

In [54]:
DICT[ROUND_ALL_PARTICIPANTS_BITSET_KEY] = server.test()[0]

In [56]:
def utility_function(client_indexes_):
    bitset_key = ROUND_CLIENTS_BITSET(client_indexes_).bits()
    if bitset_key in DICT.keys():
        return DICT[bitset_key]
    selected_clients = [server.clients[index] for index in client_indexes_]
    models = [client.model for client in selected_clients]
    p = np.array([client.datavol for client in selected_clients])
    p = p / p.sum()
    server.model = server.aggregate(models=models, p=p)
    server.model.to(CUDA)
    acc = server.test()[0]
    DICT[bitset_key] = acc
    return acc

In [77]:
EDGES = list()
for u in ROUND_PARTICIPANTS_INDEXES:
    for v in ROUND_PARTICIPANTS_INDEXES:
        if u >= v:
            continue
        w = utility_function([u]) + utility_function([v]) - utility_function([u, v])
        w *= len(server.test_data)
        w = int(np.round(w))
        EDGES.append((u, v, w))

In [86]:
G = nx.Graph()
G.add_weighted_edges_from(EDGES)
G.graph['edge_weight_attr'] = 'weight'
all_nodes = np.array(G.nodes)
all_nodes
len(G.nodes()), len(G.edges())

(16, 120)

In [98]:
cutcost, parts = metis.part_graph(G, nparts=2, recursive=False)
parts = np.array(parts)
parts_nodes = list()
for part_index in np.unique(parts):
    nodes_indexes = np.where(parts == part_index)[0]
    parts_nodes.append(all_nodes[nodes_indexes])
parts_nodes

[array([ 0,  1,  2,  3,  5,  6, 10, 11]),
 array([ 4,  7,  8,  9, 12, 13, 14, 15])]

In [103]:
exit()

: 